In [23]:
    import findspark
findspark.init()
findspark.find()

'/usr/local/spark/python/pyspark'

In [24]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("yarn")
         .appName("python-testing")
         .config("spark.executor.instances", 2)
         .config("spark.executor.memory", "800m")
         .getOrCreate())
sc = spark.sparkContext
sc

<SparkContext master=yarn appName=python-testing>

In [3]:
reads = sc.textFile('hdfs:///files/salmonella/SRR15404285.fasta').filter(lambda x: not x.startswith('>')).zipWithIndex().map(lambda x: (x[1], x[0]))
reads.take(1)

[(0,
  'TGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT')]

In [4]:
import sys
sys.path.append('/home/ubuntu/.local/lib/python3.8/site-packages')
sys.path.append('/home/ubuntu/GenASM/build/lib.linux-x86_64-3.8') # For the homebrew GenASM bindings in C. These need to be added to the datanodes too.

sc.addFile('gasm.cpython-38-x86_64-linux-gnu.so')
sc.environment['PYTHONPATH'] = '/home/ubuntu/.local/lib/python3.8/site-packages'
sc.addPyFile('hbase_connector.py')

import json
import pickle
import base64

import gasm # Homebrew package
import datasketch as ds
import hbase_connector

In [5]:
pool = hbase_connector.HbaseConnection(host="datanode1", port=9090)
# Table is from run_insert.sh
lsh = ds.lsh.MinHashLSH(threshold=0.8, num_perm=128, storage_config={'type': 'hbase', 'basename': b'hbase_salmonella_pos_prefix_8', 'hbase_pool': pool}, prepickle=True)

In [10]:
def create_hash(string, nperm=128):
    mh2 = ds.MinHash(num_perm=nperm)
    for i, c in enumerate(string):
        mh2.update(i.to_bytes(8, byteorder='little') + c.encode('utf8'))
    return mh2

In [7]:
data = reads.repartition(16)
data.take(5)

[(40,
  'GTTGCTTCACAAGCGATTGCCGTTGAGCGATTCTGGCTTCGAACTGCAGGCCGCTTTGCGGCTCCTGTAGTTGGT'),
 (41,
  'TGATTCCTCAGGAGTACGCCATTGACTACCAGGAAGGCATTAAAAATCCGGTCGGTCTTTCCGGCGTGCGTATGC'),
 (42,
  'GTGTTCATAGCTAACCTCTGAATACGTCGGAAGACGTCCCGCTACGCGCGGGATGTTTCTATAATGTTACATATT'),
 (43,
  'TATCGTGCCCGACTTTGCCGCCGCTGGCGCGAGCATCATTACCTTTAACCCGGAAGCCTCCGAACATGTTGACCG'),
 (44,
  'CCCTTCCAGATTACTTTATATGGCAGGTTCCGCGACAACCGACCTTTCTAAAAAAATAGGAATAGCACATAAAAT')]

In [25]:
testset = data.take(100000)

In [26]:
import pickle
with open('testset.pickle', 'wb') as f:
    pickle.dump(testset, f)

In [8]:
def assembler(sequences):
    import sys
    sys.path.append('/home/ubuntu/.local/lib/python3.8/site-packages')
    sys.path.append('./') # For the homebrew GenASM bindings in C. These need to be added to the datanodes too.

    import json
    import pickle
    import base64

    import gasm # Homebrew package
    import datasketch as ds
    import hbase_connector
    
    from multiprocessing import Pool

    def f(x):
        matches = []
    
        pool = hbase_connector.HbaseConnection(host="localhost", port=9090)
        # Table is from run_insert.sh
        lsh = ds.lsh.MinHashLSH(threshold=0.7, num_perm=256, storage_config={'type': 'hbase', 'basename': b'hbase_salmonella_pos_prefix_7', 'hbase_pool': pool}, prepickle=True)

        def create_hash(string, nperm=256):
            mh2 = ds.MinHash(num_perm=nperm)
            for i, c in enumerate(string):
                mh2.update(i.to_bytes(8, byteorder='little') + c.encode('utf8'))
            return mh2

        for read_number, read in sequences:
            candidate_matches = lsh.query(create_hash(read))
            qualities = [(candidate, gasm.gasmAlignment(read, candidate[1], 20, 20, 4, 5, 1)) for candidate in candidate_matches]
            qualities = list(filter(lambda x: x[1][2] != '', qualities))
            qualities.sort(key=lambda x: x[1][0])
            if len(qualities) > 0:
                match = qualities[0]
                matches.append(((read_number, read), match))

        return matches
    
    with Pool(5) as p:
        return p.map(f, list(sequences))

In [9]:
matches = data.mapPartitions(assembler)
matches.take(5)

2022-04-21 14:08:43,168 WARN scheduler.TaskSetManager: Lost task 0.0 in stage 5.0 (TID 10) (datanode4 executor 4): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1650534026272_0007/container_1650534026272_0007_01_000005/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1650534026272_0007/container_1650534026272_0007_01_000005/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 417, in func
  File "/tmp/ipykernel_72793/1271205002.py", line 41, in assembler
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 364, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/usr/lib/python

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 4 times, most recent failure: Lost task 0.3 in stage 5.0 (TID 13) (datanode4 executor 4): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1650534026272_0007/container_1650534026272_0007_01_000005/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1650534026272_0007/container_1650534026272_0007_01_000005/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 417, in func
  File "/tmp/ipykernel_72793/1271205002.py", line 41, in assemb

In [8]:
dataset = data.take(1000)

In [11]:
%%time

matches = []

for read_number, read in dataset:
    candidate_matches = lsh.query(create_hash(read))
    qualities = [(candidate, gasm.gasmAlignment(read, candidate[1], 20, 20, 4, 5, 1)) for candidate in candidate_matches]
    qualities = list(filter(lambda x: x[1][2] != '', qualities))
    qualities.sort(key=lambda x: x[1][0])
    if len(qualities) > 0:
        match = qualities[0]
        matches.append(((read_number, read), match))

CPU times: user 7.36 s, sys: 646 ms, total: 8.01 s
Wall time: 1min 28s


In [12]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [16]:
testd = [0, 1, 2, 3, 4, 5, 6, 7]
list(chunks(testd, 5))

[[0, 1, 2, 3, 4], [5, 6, 7]]

In [17]:
import multiprocessing

In [21]:
q = multiprocessing.Queue()

In [22]:
q.qsize()

0

In [19]:
import pickle
import base64
with open('testset.pickle', 'rb') as f:
    dataset = pickle.load(f)
    print(dataset[0])
    
    


(40, 'GTTGCTTCACAAGCGATTGCCGTTGAGCGATTCTGGCTTCGAACTGCAGGCCGCTTTGCGGCTCCTGTAGTTGGT')


In [33]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))   

In [35]:
import os
#os.mkdir('testdata.hadoop')

cchks = chunks(dataset, len(dataset)//32)
for i, chunk in enumerate(cchks):
    with open(f"testdata.hadoop/part-{i}", 'w') as ff:
        for item in chunk:
            pickled = pickle.dumps(item)
            base64d = base64.b64encode(pickled)
            ff.write(f"{base64d.decode('utf8')}\n")

In [5]:
t = ["b'gASVVAAAAAAAAABNCz6MS1RUR0dBVEdHVEdBVEFBVEdUR0NHQ0NBQ0dHVFRUR1RHQ0NHVEdBVFRUQUdHQ1RUVEFHQ0dBVEdDQ0dBVENHQ0NBR0dBR0FBVEFUQ5SGlC4='", "b'gASVVAAAAAAAAABNDD6MS0dHQ0FUQ0NUR0NDR0dHQ0FBQVRUR0FUVENBQ0FUR0NUQUFBVENUR0FUR0NHVFRUVEFBVFRUQ0FBVEdUVEFHR1RUVEFUVFRDVEdUR5SGlC4='", "b'gASVVAAAAAAAAABNDT6MS0dHQ0FHVFRHQ1RDQ0dDQVRHQUNHQ0NBR0dBQUdDR0FUQVRDQ0FDQ0dHR0NHQ0NDQ0FHQ0dDQ0NHQ0dDQUdDQVRDR0FHQUdDQ0dDQZSGlC4='", "b'gASVVAAAAAAAAABNDj6MS0dDR0dBVFRUQ0FBQ0NBQ0dUQ0dDQ0dBVEFDQ0dDQVRUQ0dUVEdUVENUQ0dUQ0FUR1RBQ0dUR0NBVFRUVEdHVENHVEFDR0NUVEdBVJSGlC4='", "b'gASVVAAAAAAAAABNDz6MS0NBQUNHVENBR0dDR0dUVENDR0FDVENUR0NHQ0dDVEdBQ0FBR0NDR0NUR0dUR0dHVEFDQ0dHVEFUR0dBQUNHVEdDVEdUQ0dDQ0dUQ5SGlC4='"]

In [26]:
len(dataset)/32

3125.0

In [20]:
with open('testdata.hadoop', 'r') as ff:
    str_dat = ff.readlines()

In [22]:
str_dat[0]

'gASVUwAAAAAAAABLKIxLR1RUR0NUVENBQ0FBR0NHQVRUR0NDR1RUR0FHQ0dBVFRDVEdHQ1RUQ0dBQUNUR0NBR0dDQ0dDVFRUR0NHR0NUQ0NUR1RBR1RUR0dUlIaULg==\n'

In [11]:
data_b = [base64.b64decode(d) for d in t]
data_p = [pickle.loads(b) for b in data_b]

UnpicklingError: invalid load key, 'n'.

In [2]:
data_b

NameError: name 'data_b' is not defined

In [4]:
from multiprocessing import Process, Queue

In [5]:
def f(args, q):
    try:
        raise ValueError('hello')
    except BaseException as e:
        q.put(e)
    return args

q = Queue()
p = Process(target=f, args=("hello",q), daemon=True)
p.start()
q.get()

ValueError('hello')

In [3]:
p.stop()

AttributeError: 'Process' object has no attribute 'stop'